In [1]:
import cv2
import argparse
import imutils
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from imutils.video import VideoStream

In [2]:
MODEL_DIR = '../../model/'
IMG_HEIGHT = 256
IMG_WIDTH = 256
CLASS_NAMES = ['correctly-masked', 'not-masked', 'uncorrectly-masked']
CORRECTLY_MASKED = CLASS_NAMES[0]
NOT_MASKED = CLASS_NAMES[1]
INCORRECTLY_MASKED = CLASS_NAMES[2]

In [3]:
model = keras.models.load_model(MODEL_DIR)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0

In [4]:
def get_face(img, x, y, w, h) -> Image.Image: 
    crop_img = img[y:y+h, x:x+w]
    cv2.imshow("cropped", crop_img)
    # Convert image from GBR to RGB
    rgb_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(rgb_img)
    newsize = (IMG_WIDTH, IMG_HEIGHT)
    resized_img = pil_img.resize(newsize)
    return resized_img

def predict(resized_img):
    img_array = keras.preprocessing.image.img_to_array(resized_img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    prediction = CLASS_NAMES[np.argmax(score)]
    confidence = 100 * np.max(score)
    return prediction, confidence
    
def get_color(prediction):
    green = (36,255,12)
    red = (0,0,255)
    yellow = (0,255,255)
    if prediction == CORRECTLY_MASKED:
        color = green
    elif prediction == NOT_MASKED:
        color = red
    else:
        color = yellow

    return color

In [5]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
print("[INFO] starting video stream ...")
cap = cv2.VideoCapture(0)
_, img = cap.read()
if (cap.isOpened() == False):
    print("[ERROR] Unable to read camera feed ...")

while True:
    _, img = cap.read()
    # Convert to grayscale (haarcascade works with grayscale)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        resized_img = get_face(img, x, y, w, h)
        prediction, confidence= predict(resized_img)
        color = get_color(prediction)
        print(f'{prediction}: {confidence}')
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, str(prediction), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        
    cv2.imshow('img', img)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
exit()

[INFO] starting video stream ...
uncorrectly-masked: 68.0630624294281
not-masked: 99.04879927635193
not-masked: 99.57077503204346
not-masked: 98.55818152427673
not-masked: 98.15537929534912
not-masked: 98.9238977432251
not-masked: 98.96089434623718
not-masked: 98.27719330787659
not-masked: 98.91468286514282
not-masked: 99.19126033782959
not-masked: 99.67832565307617
not-masked: 53.832608461380005
not-masked: 93.21887493133545
not-masked: 97.9516327381134
not-masked: 95.3223168849945
not-masked: 97.79127836227417
not-masked: 98.54904413223267
not-masked: 99.82364177703857
not-masked: 99.25893545150757
not-masked: 99.0626573562622
not-masked: 98.4835684299469
not-masked: 97.84020781517029
not-masked: 63.03759217262268
not-masked: 84.92312431335449
not-masked: 94.93793845176697
not-masked: 96.86786532402039
not-masked: 90.7029390335083
not-masked: 74.92262721061707
uncorrectly-masked: 52.03178524971008
not-masked: 77.1487832069397
not-masked: 97.62923121452332
not-masked: 62.3461484909057